In [1]:
!pip install box2d-py
!pip install gym[Box_2D]

In [0]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import gym
import os


In [0]:
class ActorCritic(nn.Module):
    def __init__(self, num_actions):
        super().__init__()
        self.fc2 = nn.Linear(8, 2048)
        self.fc3 = nn.Linear(2048, 512)
        self.pi = nn.Linear(512, num_actions)
        self.v = nn.Linear(512, 1)
        
    def forward(self, x):
#         x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pi = self.pi(x)
        v = self.v(x)
        return pi, v
        

In [0]:
class Agent():
    def __init__(self):
        self.gamma = 0.99
        self.log_probs = None
        self.env = gym.make('LunarLander-v2')
        num_actions = self.env.action_space.n
        self.ac_network = ActorCritic(num_actions=num_actions).cuda()
        self.MODEL_PATH = '/content/ac_model.pth'
        if os.path.exists(self.MODEL_PATH):
            print('Existing model found!')
            self.ac_network.load_state_dict(torch.load(self.MODEL_PATH))
            self.ac_network.eval()
        else:
            print('No existing model.')
        self.optimizer = torch.optim.Adam(self.ac_network.parameters(), lr=1e-5)
        self.MAX_FRAMES = 10000
        self.NUM_EPISODES = 3000
        
    def choose_action(self, state):
        policy, _ = self.ac_network(torch.tensor(state).cuda())
        probablities = F.softmax(policy)
        action_distribution = torch.distributions.Categorical(probablities)
        action = action_distribution.sample()
        self.log_probs = action_distribution.log_prob(action)
        return action.item()
    
    def learn(self, state, next_state, reward, done):
#         with torch.no_grad():
        self.optimizer.zero_grad()
        _, curr_value = self.ac_network(torch.tensor(state).cuda())
        _, next_value = self.ac_network(torch.tensor(next_state).cuda())
        reward = torch.tensor(reward).cuda()
        
        if done:
            advantage = reward - curr_value
        else:
            advantage = reward + self.gamma * next_value - curr_value
            
        actor_loss = -self.log_probs * advantage
        critic_loss = advantage ** 2
        
        loss = actor_loss + critic_loss
#         loss.item()
        loss.backward()
        self.optimizer.step()
        return loss.item()
        
    def play(self):
        for i_episode in range(self.NUM_EPISODES):
            observation = self.env.reset()
            curr_state = observation
            done = False
            total_rewards = 0
            total_loss = 0
            num_frames = 0
            while not done:
                action = self.choose_action(curr_state)
                observation, reward, done, info = self.env.step(action)
                loss = self.learn(state=curr_state, next_state=observation, reward=reward, done=done)
                curr_state = observation
                total_rewards += reward
                total_loss += loss
                num_frames += 1
                
            print(f'Episode # {i_episode} done, total reward: {total_rewards}, loss: {total_loss/num_frames}')
            
            
            
        self.env.close()
                
        

In [19]:
agent = Agent()
agent.play()

No existing model.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Episode # 0 done, total reward: -156.68605943428054, loss: 104.66955067961494
Episode # 1 done, total reward: -100.45328449626248, loss: 139.7136024533937
Episode # 2 done, total reward: -289.3133845842089, loss: 115.38166632128879
Episode # 3 done, total reward: -87.269329699724, loss: 190.0127173672944
Episode # 4 done, total reward: -150.75573221690325, loss: 110.83326312445439
Episode # 5 done, total reward: -471.1201985264931, loss: 113.22605900444529
Episode # 6 done, total reward: -338.0007291601403, loss: 77.18927843626449
Episode # 7 done, total reward: -93.24963535003518, loss: 124.11733922136149
Episode # 8 done, total reward: -340.8616220538005, loss: 114.92454602643848
Episode # 9 done, total reward: -125.72967414698493, loss: 93.46061236176777
Episode # 10 done, total reward: -32.03918514502446, loss: 152.75688234724154
Episode # 11 done, total reward: -364.38930491960923, loss: 79.89771149280068
Episode # 12 done, total reward: -38.72470460558479, loss: 127.8249586747959

KeyboardInterrupt: ignored

In [14]:
print(agent.env.action_space.n)

4


In [0]:
torch.save(agent.ac_network.state_dict(), agent.MODEL_PATH)